# Scale-invariant Feature Transform (SIFT) Demo

This demonstration will illustrate how we can determine keypoints for local features with SIFT.  In addition, we will use these keypoints to match these features between two images. This example uses a version of OpenCV that includes a version of the SIFT algorithm.

If you want to learn more about the algorithm, you can read the updated paper from David G. Lowe (link below).

**Original Paper** - [Distinctive Image Featuresfrom Scale-Invariant Keypoints](https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf)

## Install

We will install the following modules that will be used throughout this notebook. Execute the following cell to install the needed dependencies.

In [ ]:
!pip install opencv-contrib-python==3.3.0.9

## Imports & Utility Functions

Next, we will need to import libraries and add a utility function that we will use throughout the notebook:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv

# This is a utility function that we will use to display images throughout the notebook
def showImage(cv_image, isGray = False, title = ''):
    plt.figure(figsize=(20,20))
    if isGray:
        plt.imshow(cv_image, cmap = plt.cm.gray)
    else:
        plt.imshow(cv_image)
    if title:
        plt.title(title)
    plt.show()

## Our Objective

We'll be working with the image of guitars that has been used throughout the course. We will first explore keypoint detection.  Next, we will isolate our efforts on the body of one of the guitars and use these keypoints to match across images.

You can see the original image below:

![Guitar Image](guitars1.jpg)

## Detecting Keypoints with SIFT

First, we will need to examine how SIFT calculates keypoints for the entire image.  To accomplish this, we will first need to convert our image to grayscale.

In [ ]:
original_image = cv.imread('guitars1.jpg')
gray_image = cv.cvtColor(original_image,cv.COLOR_BGR2GRAY)
showImage(gray_image, True, 'Grayscale version of original image')

Our first step in detecting keypoints will be to create the SIFT class instance that we will use to detect keypoints and compute feature descriptors:

In [ ]:
# SIFT Configuration Parameters
max_features = 0 # If zero, there is no limit
octaves = 5
contrastThreshold = 0.04
edgeThreshold = 10
sigma = 1.6

# Create SIFT detector
sift = cv.xfeatures2d.SIFT_create(max_features, octaves, contrastThreshold, edgeThreshold, sigma)

After we have instantiated the SIFT class, we can now detect keypoints within the grayscale version of the image. We will output the number of keypoints that are detected:

In [ ]:
# Detect keypoints from our image
keypoints = sift.detect(gray_image,None)

# Output Number of Keypoints and Image with Keypoints Highlighted
print(f'SIFT Keypoints: {len(keypoints)}')

OpenCV provides a utility for visualizing the keypoints that have been detected.  By utilizing the flag for rich keypoints, we can see the orientation and magnitude of each keypoint.  Given the size of the image and number of keypoints, it may be difficult to discern each of the individual keypoints.

In [ ]:
keypoints_image = cv.drawKeypoints(gray_image, keypoints, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
showImage(keypoints_image, False, 'Keypoints in original image')

We will be working to define features for the middle guitar in the image. We will start by isolating that guitar's body in our source image.

In [ ]:
guitar_body_image = cv.cvtColor(cv.imread('guitar.jpg'), cv.COLOR_BGR2GRAY)
showImage(guitar_body_image, True, 'Guitar Body Image')

Next, we can detect the keypoints for this image:

In [ ]:
# Detect keypoints
guitar_keypoints = sift.detect(guitar_body_image, None)

# Output Number of Keypoints and Image with Keypoints Highlighted
print(f'SIFT Keypoints: {len(guitar_keypoints)}')
guitar_keypoints_image = cv.drawKeypoints(guitar_body_image, guitar_keypoints, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
showImage(guitar_keypoints_image, False, 'Guitar Body Keypoints')

## Matching Across Images

With these keypoints defined for the guitar body, we will now test the capabilities of SIFT's feature descriptor for matching the guitar body across images.

To accomplish this, we will be using OpenCV's Brute Force Matcher, `BFMatcher`. In addition, we will want to apply a filter mechanism to make sure we only consider the best matches out of the results.  For this, we will be using SIFT's ratio test which was detailed by Lowe in the original paper.  To do this, we will use the k-nearest neighbor match of the `BFMatcher` class (with the `k` set to 2).  We will then filter the first matches that have a distance less than `0.75` of the second match.

In [ ]:
# This function will get a grayscale version of the image, keypoints, and keypoint descriptors
def readImage(image_filename):
    image = cv.cvtColor(cv.imread(image_filename), cv.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return image, keypoints, descriptors
    
# This function will use a brute force matcher with the SIFT descriptors to find matches between the images
def matchImage(image1_filename, image2_filename, title):
    # Get keypoints and descriptors for images
    image1, image1_keypoints, image1_descriptors = readImage(image1_filename)
    image2, image2_keypoints, image2_descriptors = readImage(image2_filename)
    
    # Create the Brute Force Matcher and Match using knn
    bf = cv.BFMatcher()
    matches = bf.knnMatch(image1_descriptors, image2_descriptors, k=2)
    
    # Filter out only good matches based on SIFT ratio test
    good_matches = [[m1] for m1,m2 in matches if m1.distance < 0.75 * m2.distance]
    print(f'{len(matches)} All Matches')
    print(f'{len(good_matches)} Matches Passing Ratio Test')
    
    # Draw the matches
    output_image = cv.drawMatchesKnn(image1,
                                     image1_keypoints,
                                     image2,
                                     image2_keypoints,
                                     good_matches,
                                     None,
                                     flags=2)
    
    # Show the image
    showImage(output_image, False, title)

With these functions in place, we will first match the guitar body image against the original image of all three guitars:

In [ ]:
matchImage('guitar.jpg', 'guitars1.jpg', 'Matching the Guitar Body Against the Original Image')

To illustrate the rotation-invariance of SIFT, we will test it against two rotated versions of the image:

In [ ]:
matchImage('guitar.jpg', 'guitars2.jpg', 'Matching against 180 Degree Rotation')

In [ ]:
matchImage('guitar.jpg', 'guitars3.jpg', 'Matching against 215 Degree Rotation')

Next, we will illustrate matching with a rotated and scaled version of the original image to illustrate both rotation-invariance and scale-invariance:

In [ ]:
matchImage('guitar.jpg', 'guitars4.jpg', 'Matching against 180 Rotation and Smaller Scale')

With these descriptors in place, we can now also test against a completely different image of the guitar body:

In [ ]:
matchImage('guitar.jpg', 'guitar-match2.jpg', 'Matching against Different Image of Guitar Body')